In [3]:
from pystac_client import Client
from odc.stac import load

In [4]:
client=Client.open("https://earth-search.aws.element84.com/v1")

In [66]:
collection= "sentinel-2-l2a"
baton_rouge_bbox = [-91.2500, 30.3500, -90.9000, 30.5500]
jefferson_county_tx = [-94.4300, 29.5900, -93.7400, 30.2400]
galveston = [-95.0000, 29.1500, -94.7000, 29.3800]

date="2019-09-01/2019-10-01"
cloud_cover = ("0", "75")

In [68]:
location = 'galveston'

In [67]:
search = client.search(collections=[collection],bbox=galveston , datetime=date, query=[
        f"eo:cloud_cover<{cloud_cover[1]}",
        f"eo:cloud_cover>{cloud_cover[0]}"
    ])

In [69]:
print(f"Number of images are {search.matched()}")

for item in search.items():
    print(item.id)

Number of images are 20
S2A_15RTN_20191001_0_L2A
S2A_15RUN_20191001_0_L2A
S2B_15RTN_20190929_0_L2A
S2B_15RUN_20190929_0_L2A
S2B_15RTN_20190926_0_L2A
S2B_15RUN_20190926_0_L2A
S2A_15RTN_20190924_0_L2A
S2A_15RUN_20190924_0_L2A
S2A_15RTN_20190921_0_L2A
S2A_15RUN_20190921_0_L2A
S2A_15RTN_20190914_0_L2A
S2A_15RUN_20190914_0_L2A
S2A_15RTN_20190911_0_L2A
S2A_15RUN_20190911_0_L2A
S2B_15RTN_20190909_0_L2A
S2B_15RUN_20190909_0_L2A
S2B_15RTN_20190906_0_L2A
S2B_15RUN_20190906_0_L2A
S2A_15RTN_20190901_0_L2A
S2A_15RUN_20190901_0_L2A


In [70]:
data =load(search.items(),bbox=galveston , groupby = "solar_day", chunks ={})
data

<xarray.Dataset> Size: 5GB
Dimensions:       (y: 2596, x: 2956, time: 10)
Coordinates:
  * y             (y) float64 21kB 3.252e+06 3.252e+06 ... 3.226e+06 3.226e+06
  * x             (x) float64 24kB 3.055e+05 3.055e+05 ... 3.35e+05 3.35e+05
    spatial_ref   int32 4B 32615
  * time          (time) datetime64[ns] 80B 2019-09-01T17:05:37.646000 ... 20...
Data variables: (12/32)
    aot           (time, y, x) uint16 153MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>
    blue          (time, y, x) uint16 153MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>
    coastal       (time, y, x) uint16 153MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>
    green         (time, y, x) uint16 153MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>
    nir           (time, y, x) uint16 153MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>
    nir08         (time, y, x) uint16 153MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>
    ...            ...
    rededge3-jp2  (time, y, x) uint16 153MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>
    scl-jp2       (time, y, x) uint8 77MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>
    swir16-jp2    (time, y, x) uint16 153MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>
    swir22-jp2    (time, y, x) uint16 153MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>
    visual-jp2    (time, y, x) float32 307MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>
    wvp-jp2       (time, y, x) uint16 153MB dask.array<chunksize=(1, 2596, 2956), meta=np.ndarray>

In [71]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import numpy as np

numb_days= data[["red"]].time.size

for t in range(0,numb_days):
  rgb_image = data[["red","green","blue"]].isel(time=t).to_array()
  dt = pd.to_datetime(rgb_image.time.values)
  
  year = dt.year
  month = dt.month
  day = dt.day

  # Double digit for month and day
  if month<10:
    month=f'0{month}'
  if day<10:
    day=f'0{day}'

  date_string= f"{year}_{month}_{day}"
  date_string_title= f"{month}/{day}/{year}"

  count1 = np.count_nonzero(rgb_image[1,:,:])
  count2 = np.size(rgb_image[1,:,:])
  ratio= count1/count2

  # Print Date and Coverage 
  print(f"Date:{date_string}, Coverage:{format(ratio*100,'.2f')}%")

  if ratio>0.7:
    # Plot the RGB image
    fig, ax = plt.subplots()
    rgb_image.plot.imshow(robust=True)
    plt.title(f"{location}_{date_string_title} by Sentinel-2")
    plt.savefig(f"../figures/satellite_images/{location}_{date_string}.png")
    plt.close(fig)

Date:2019_09_01, Coverage:100.00%
Date:2019_09_06, Coverage:100.00%
Date:2019_09_09, Coverage:0.29%
Date:2019_09_11, Coverage:100.00%
Date:2019_09_14, Coverage:0.35%
Date:2019_09_21, Coverage:100.00%
Date:2019_09_24, Coverage:0.13%
Date:2019_09_26, Coverage:100.00%
Date:2019_09_29, Coverage:0.64%
Date:2019_10_01, Coverage:100.00%
